In [36]:
import backtrader as bt
import pandas as pd
from datetime import datetime

class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
    )
    def log(self, txt, dt=None):
        ''' 记录'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # 增加移动平均指标
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
        
        # 增加划线的指标
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        # Write down: no pending order
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        # flag=stock_hfq_df.loc[str(self.datas[0].datetime.date(0))].Value_pre>stock_hfq_df.loc[str(self.datas[0].datetime.date(-1))].Value_pre
        flag=stock_hfq_df.loc[str(self.datas[0].datetime.date(0))].close>stock_hfq_df.loc[str(self.datas[0].datetime.date(-1))].close
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...SMA均线
            if self.dataclose[0] > self.sma[0] and flag:
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                
                   
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.sma[0] and not flag:#SMA均线
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [37]:
#获取数据
import datetime
# stock_hfq_df = pd.read_excel("PRE_BCHAIN.xlsx",index_col='Date',parse_dates=True)
stock_hfq_df = pd.read_excel("Bitcoin.xlsx",index_col='date',parse_dates=True)
stock_hfq_df = stock_hfq_df[0:30]
start_date = datetime.datetime(2016, 10, 21)  # 回测开始时间
end_date = datetime(2021, 09, 10)  # 回测结束时间
# end_date = start_date + datetime.timedelta(days=30)
# pre=stock_hfq_df.close
# Close	Low	High	Open	Volume

In [38]:
cerebro = bt.Cerebro()
# 增加一个策略
cerebro.addstrategy(TestStrategy)
# 将数据传入回测系统
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)
cerebro.broker.setcash(1000)

cerebro.broker.setcommission(commission=0.02)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 1000.00


IndexError: array assignment index out of range